In [20]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import torch
from mmcv import Config
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
configCAM = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
cfg = Config.fromfile(configCAM)

In [4]:
cfg.data.test

{'type': 'CompCars',
 'data_prefix': 'data/CompCars_sv_original_split/val',
 'ann_file': 'data/CompCars_sv_original_split/meta/val.txt',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'Resize', 'size': (256, -1)},
  {'type': 'CenterCrop', 'crop_size': 224},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'ImageToTensor', 'keys': ['img']},
  {'type': 'Collect', 'keys': ['img']}]}

In [31]:
a = np.random.rand(3,5,10)
i = a <= 0.3
b = np.copy(a)
b[i] = 0

In [32]:
a

array([[[0.37185827, 0.76462865, 0.4307922 , 0.52129781, 0.90677901,
         0.1308136 , 0.89102806, 0.91783943, 0.36541535, 0.08829013],
        [0.4527965 , 0.18349389, 0.62848587, 0.3794151 , 0.44687261,
         0.10237352, 0.10539415, 0.32135222, 0.5952298 , 0.29907406],
        [0.41202144, 0.89513119, 0.7420808 , 0.72819878, 0.8778329 ,
         0.78521323, 0.66546102, 0.45277554, 0.28962259, 0.81702973],
        [0.20770689, 0.11743793, 0.08541892, 0.68808841, 0.41111316,
         0.50775082, 0.05161047, 0.80362897, 0.23960035, 0.46100641],
        [0.48374921, 0.08740086, 0.69949978, 0.75778382, 0.03854761,
         0.00889882, 0.98753314, 0.25635819, 0.86203818, 0.18778676]],

       [[0.68791487, 0.46991184, 0.00988849, 0.11646755, 0.95009213,
         0.78421666, 0.42238011, 0.60807175, 0.30036339, 0.68614283],
        [0.94933333, 0.41986046, 0.89085252, 0.05061708, 0.47103807,
         0.90954247, 0.42618682, 0.03199175, 0.83517199, 0.39765098],
        [0.68706244, 0.66

In [33]:
b

array([[[0.37185827, 0.76462865, 0.4307922 , 0.52129781, 0.90677901,
         0.        , 0.89102806, 0.91783943, 0.36541535, 0.        ],
        [0.4527965 , 0.        , 0.62848587, 0.3794151 , 0.44687261,
         0.        , 0.        , 0.32135222, 0.5952298 , 0.        ],
        [0.41202144, 0.89513119, 0.7420808 , 0.72819878, 0.8778329 ,
         0.78521323, 0.66546102, 0.45277554, 0.        , 0.81702973],
        [0.        , 0.        , 0.        , 0.68808841, 0.41111316,
         0.50775082, 0.        , 0.80362897, 0.        , 0.46100641],
        [0.48374921, 0.        , 0.69949978, 0.75778382, 0.        ,
         0.        , 0.98753314, 0.        , 0.86203818, 0.        ]],

       [[0.68791487, 0.46991184, 0.        , 0.        , 0.95009213,
         0.78421666, 0.42238011, 0.60807175, 0.30036339, 0.68614283],
        [0.94933333, 0.41986046, 0.89085252, 0.        , 0.47103807,
         0.90954247, 0.42618682, 0.        , 0.83517199, 0.39765098],
        [0.68706244, 0.66

In [75]:
c = torch.rand(3,5,5)

In [76]:
c

tensor([[[0.8690, 0.0161, 0.6682, 0.9861, 0.0021],
         [0.6235, 0.2898, 0.0241, 0.2157, 0.5477],
         [0.8950, 0.6189, 0.3978, 0.4093, 0.7588],
         [0.5049, 0.4341, 0.5237, 0.6691, 0.8020],
         [0.5957, 0.0660, 0.7846, 0.2897, 0.1322]],

        [[0.9284, 0.2669, 0.6066, 0.5173, 0.9353],
         [0.6283, 0.4970, 0.8394, 0.9177, 0.5669],
         [0.4467, 0.4414, 0.2630, 0.2332, 0.2946],
         [0.5432, 0.2607, 0.1558, 0.8925, 0.3728],
         [0.5690, 0.4673, 0.9616, 0.0369, 0.1743]],

        [[0.5335, 0.2750, 0.2771, 0.2014, 0.3505],
         [0.3916, 0.1711, 0.9827, 0.9704, 0.1586],
         [0.1201, 0.7778, 0.8246, 0.9393, 0.6587],
         [0.0436, 0.1861, 0.0162, 0.9861, 0.4864],
         [0.9151, 0.7858, 0.9623, 0.9134, 0.0597]]])

In [55]:
c[c<.3] = 0

In [77]:
c

tensor([[[0.8690, 0.0161, 0.6682, 0.9861, 0.0021],
         [0.6235, 0.2898, 0.0241, 0.2157, 0.5477],
         [0.8950, 0.6189, 0.3978, 0.4093, 0.7588],
         [0.5049, 0.4341, 0.5237, 0.6691, 0.8020],
         [0.5957, 0.0660, 0.7846, 0.2897, 0.1322]],

        [[0.9284, 0.2669, 0.6066, 0.5173, 0.9353],
         [0.6283, 0.4970, 0.8394, 0.9177, 0.5669],
         [0.4467, 0.4414, 0.2630, 0.2332, 0.2946],
         [0.5432, 0.2607, 0.1558, 0.8925, 0.3728],
         [0.5690, 0.4673, 0.9616, 0.0369, 0.1743]],

        [[0.5335, 0.2750, 0.2771, 0.2014, 0.3505],
         [0.3916, 0.1711, 0.9827, 0.9704, 0.1586],
         [0.1201, 0.7778, 0.8246, 0.9393, 0.6587],
         [0.0436, 0.1861, 0.0162, 0.9861, 0.4864],
         [0.9151, 0.7858, 0.9623, 0.9134, 0.0597]]])

In [82]:
c.argmax(dim=2)

tensor([[3, 0, 0, 4, 2],
        [4, 3, 0, 3, 2],
        [0, 2, 3, 3, 2]])

In [86]:
c.max(dim=2)

torch.return_types.max(
values=tensor([[0.9861, 0.6235, 0.8950, 0.8020, 0.7846],
        [0.9353, 0.9177, 0.4467, 0.8925, 0.9616],
        [0.5335, 0.9827, 0.9393, 0.9861, 0.9623]]),
indices=tensor([[3, 0, 0, 4, 2],
        [4, 3, 0, 3, 2],
        [0, 2, 3, 3, 2]]))

In [87]:
m = c.max(dim=2)

In [88]:
m.values

tensor([[0.9861, 0.6235, 0.8950, 0.8020, 0.7846],
        [0.9353, 0.9177, 0.4467, 0.8925, 0.9616],
        [0.5335, 0.9827, 0.9393, 0.9861, 0.9623]])

In [89]:
m.values < 0.7

tensor([[False,  True, False, False, False],
        [False, False,  True, False, False],
        [ True, False, False, False, False]])

In [90]:
m.indices[m.values < 0.7] = -1

In [91]:
m

torch.return_types.max(
values=tensor([[0.9861, 0.6235, 0.8950, 0.8020, 0.7846],
        [0.9353, 0.9177, 0.4467, 0.8925, 0.9616],
        [0.5335, 0.9827, 0.9393, 0.9861, 0.9623]]),
indices=tensor([[ 3, -1,  0,  4,  2],
        [ 4,  3, -1,  3,  2],
        [-1,  2,  3,  3,  2]]))

In [100]:
a = ('test','abc','nani')
b = ('new')

In [99]:
a

('test', 'abc', 'nani')

In [95]:
a.index('test')

0

In [103]:
n = a + (b)

TypeError: can only concatenate tuple (not "str") to tuple

In [102]:
b

'new'